In [1]:
#Dependecies
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import datetime
from sqlalchemy.orm import sessionmaker

In [2]:
#Read login credentials from external file.
with open('C:\\Users\\Marcus\\Desktop\\credentials.txt','r') as creds:
    all_lines = creds.readlines()

In [3]:
#Create engine to different databases
engines = {
    'classicmod': create_engine(all_lines[0] + '/classicmodels'),
    'classicware': create_engine(all_lines[0] + '/classicDWH')
}

In [4]:
#Create connection to databases
conn = engines['classicmod'].connect()
conndw = engines['classicware'].connect()

# Extract and Transform

In [5]:
#Create Employee Dimension 
dim_emp = pd.read_sql_query('select employeeNumber,lastName,firstName,officeCode \
                            from employees',conn)

In [6]:
dim_emp.head()

,employeeNumber,lastName,firstName,officeCode
0,1002,Murphy,Diane,1
1,1056,Patterson,Mary,1
2,1076,Firrelli,Jeff,1
3,1088,Patterson,William,6
4,1102,Bondur,Gerard,4


In [7]:
#Create Office Dimension 
dim_office = pd.read_sql_query('select officeCode,city,state,country \
                                from offices',conn)

In [8]:
dim_office

,officeCode,city,state,country
0,1,San Francisco,CA,USA
1,2,Boston,MA,USA
2,3,NYC,NY,USA
3,4,Paris,None,France
4,5,Tokyo,Chiyoda-Ku,Japan
5,6,Sydney,None,Australia
6,7,London,None,UK


In [9]:
#Create product Dimension
dim_product = pd.read_sql_query('select productCode,productName,products.productLine from products \
                            inner join productlines \
                            on productlines.productline=products.productline;',conn)

In [10]:
dim_product.head()

,productCode,productName,productLine
0,S10_1949,1952 Alpine Renault 1300,Classic Cars
1,S10_4757,1972 Alfa Romeo GTA,Classic Cars
2,S10_4962,1962 LanciaA Delta 16V,Classic Cars
3,S12_1099,1968 Ford Mustang,Classic Cars
4,S12_1108,2001 Ferrari Enzo,Classic Cars


In [11]:
#Create customer Dimension
dim_customer = pd.read_sql_query('select customerNumber,customerName,contactLastName, \
                                   contactFirstName, city, country \
                                   from Customers',conn)

In [12]:
dim_customer.head()

,customerNumber,customerName,contactLastName,contactFirstName,city,country
0,103,Atelier graphique,Schmitt,Carine,Nantes,France
1,112,Signal Gift Stores,King,Jean,Las Vegas,USA
2,114,"Australian Collectors, Co.",Ferguson,Peter,Melbourne,Australia
3,119,La Rochelle Gifts,Labrune,Janine,Nantes,France
4,121,Baane Mini Imports,Bergulfsen,Jonas,Stavern,Norway


In [13]:
#Create Order Dimension
dim_order = pd.read_sql_query('select orders.orderNumber, status \
                                from orders \
                                inner join orderdetails \
                                on orders.orderNumber=orderdetails.orderNumber;',conn)

In [14]:
dim_order.head()

,orderNumber,status
0,10100,Shipped
1,10100,Shipped
2,10100,Shipped
3,10100,Shipped
4,10101,Shipped


In [15]:
#Create dim_Date
dim_date = pd.DataFrame(pd.date_range('01/01/2003', end='31/12/2005'),columns=['Dates'])
dim_date['Date_key'] = dim_date['Dates'].apply(lambda x: x.strftime('%Y%m%d'))
dim_date['Day_of_Week'] = dim_date['Dates'].dt.weekday_name
dim_date['Month'] = dim_date['Dates'].dt.month
dim_date['Week'] = dim_date['Dates'].dt.week
dim_date['Day'] = dim_date['Dates'].dt.day

In [16]:
dim_date.head()

,Dates,Date_key,Day_of_Week,Month,Week,Day
0,2003-01-01,20030101,Wednesday,1,1,1
1,2003-01-02,20030102,Thursday,1,1,2
2,2003-01-03,20030103,Friday,1,1,3
3,2003-01-04,20030104,Saturday,1,1,4
4,2003-01-05,20030105,Sunday,1,1,5


In [17]:
#Create Fact_Table
Fact_Measure = pd.read_sql_query('select orderdetails.quantityOrdered, orderdetails.priceEach, employees.employeeNumber, \
                                    products.productCode, customers.customerNumber,orders.orderNumber, orders.orderDate, \
                                    orders.requiredDate \
                                    from employees \
                                    inner join offices \
                                    on offices.officeCode = employees.officeCode \
                                    inner join Customers \
                                    on employees.employeeNumber=Customers.salesRepEmployeeNumber \
                                    inner join payments \
                                    on Customers.customerNumber=payments.customerNumber \
                                    inner join orders \
                                    on Customers.customerNumber=orders.customerNumber \
                                    inner join orderdetails \
                                    on orders.orderNumber=orderdetails.orderNumber \
                                    inner join products \
                                    on orderdetails.productCode=products.productCode \
                                    inner join productlines \
                                    on products.productLine=productlines.productLine;',conn)
Fact_Measure['Order_Date_key'] = Fact_Measure['orderDate'].apply(lambda x: x.strftime('%Y%m%d'))
Fact_Measure['Required_Date_key'] = Fact_Measure['requiredDate'].apply(lambda x: x.strftime('%Y%m%d'))

Fact_Measure = Fact_Measure.drop(['orderDate','requiredDate'],axis=1)

In [18]:
Fact_Measure.head()

,quantityOrdered,priceEach,employeeNumber,productCode,customerNumber,orderNumber,Order_Date_key,Required_Date_key
0,30,136.00,1216,S18_1749,363,10100,20030106,20030113
1,50,55.09,1216,S18_2248,363,10100,20030106,20030113
2,22,75.46,1216,S18_4409,363,10100,20030106,20030113
3,49,35.29,1216,S24_3969,363,10100,20030106,20030113
4,30,136.00,1216,S18_1749,363,10100,20030106,20030113


# Load

In [20]:
#Load all transformed tables to Data warehouse Mysql
dim_emp.to_sql(name='dim_emp',con=conndw)
dim_office.to_sql(name='dim_office',con=conndw)
dim_product.to_sql(name='dim_product',con=conndw)
dim_customer.to_sql(name='dim_customer',con=conndw)
dim_order.to_sql(name='dim_order',con=conndw)
dim_date.to_sql(name='dim_date',con=conndw)
Fact_Measure.to_sql(name='Fact_Measure',con=conndw)

C:\Users\Marcus\Anaconda3\lib\site-packages\pandas\io\sql.py:1191: UserWarning: The provided table name 'Fact_Measure' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [21]:
#Close connection
conn.close()
conndw.close()